In [1]:
%config IPCompleter.use_jedi = False
from loader import load_data
import numpy as np
from sklearn.linear_model import RidgeCV
from rascal.representations import SphericalInvariants as SOAP
from rascal.utils import get_optimal_radial_basis_hypers
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_id
from skcosmo.model_selection import atom_groups_by_frame
from sklearn.linear_model import LinearRegression, Ridge
from copy import deepcopy
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

Original error:
No module named 'sklearn.utils._tags'


In [33]:
train_structures, test_structures, train_properties, test_properties = load_data("./make_tensor_data/train_tensor/CSD-3k+S546_shift_tensors.xyz",\
                                                                                    "./make_tensor_data/test_tensor/CSD-500+104-7_shift_tensors.xyz",selected_species=1,random_subsample_train=1000,random_subsample_test=200)

In [4]:
class BufferedFeatures:
    def __init__(self, structures, calculator_params, calculator=SOAP):
        
        self.calculator = calculator
        self.X = None
        self.structures = structures
        self.calculator_params = calculator_params

    def get_features(self, update_params):
        
        updated_params = self.calculator_params.copy()
        
        for key, value in update_params.items():
            
            if isinstance(value, np.integer):
                value = int(value)
            if isinstance(value, np.floating):
                value = float(value)
            if isinstance(value, np.ndarray):
                value = value.tolist()
                
            updated_params[key] = value

        
        if self.X is None:
            
            #print("Initial calculation")
            calculator = self.calculator(**updated_params)
            self.X = calculator.transform(self.structures).get_features(calculator)
        
        else:
            
            if updated_params == self.calculator_params:
                #print("Stored")
                pass
            else:
                #print("Recalculate")
                calculator = self.calculator(**updated_params)
                self.X = calculator.transform(self.structures).get_features(calculator)
                
                #print(self.calculator.transform(self.structures).get_features(self.calculator).shape)
        
        self.calculator_params = updated_params
        
        return self.X

In [10]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.,
              max_radial=8,
              max_angular=8,
              gaussian_sigma_constant=0.3,
              gaussian_sigma_type="Constant",
              radial_basis="GTO",
              normalize=True,
              cutoff_smooth_width=0.3,
              optimization=
                    dict(
                            Spline=dict(
                               accuracy=1.0e-05
                            )
                        ),
              compute_gradients=False,
              )

In [27]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.,
              max_radial=8,
              max_angular=8,
              gaussian_sigma_constant=0.3,
              gaussian_sigma_type="Constant",
              radial_basis="GTO",
              normalize=True,
              cutoff_smooth_width=0.3,
              optimization=
                    dict(
                            Spline=dict(
                               accuracy=1.0e-05
                            )
                        ),
              compute_gradients=False,
              )

space = [Real(10**-5, 10**2, "log-uniform", name='alpha'),
        Real(0.05,1.5, "uniform", name="gaussian_sigma_constant"),
        Real(2.,4.5, "uniform", name="interaction_cutoff")]

reg = Ridge()
y = train_properties
atom_groups = atom_groups_by_frame(train_structures)
Feature_gen = BufferedFeatures(train_structures, hypers)


@use_named_args(space)
def soap_objective(**params):
    update_dict = {}
    
    new_params = params.copy()
    
    for key, value in new_params.items():
        if key in Feature_gen.calculator_params:
            #hypers[key] = value
            update_dict[key] = params.pop(key, None)
    
    #print(update_dict)
    reg.set_params(**params)
    

    X = Feature_gen.get_features(update_dict)
    #print(X.shape)
    
    #print(Feature_gen.hypers["max_angular"])
    splits = list(GroupKFold(n_splits=5).split(X,y,groups=atom_groups))
    
    return -np.mean(cross_val_score(reg, X, y, cv=splits, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

res_gp = gp_minimize(soap_objective, space, n_calls=200, random_state=0)

KeyboardInterrupt: 

In [40]:
reg.alpha

2.043192897589359

In [5]:
def get_features_in_parallel(frames,calculator,blocksize=100,n_jobs=-1):
    """helper function that returns the features of a calculator (from calculator.transform())
       in parallel
    """
    #for np.concatenate. arrays in list should all have same shape
    hypers = calculator.hypers
    hypers["expansion_by_species_method"] = "user defined"
    hypers["global_species"] = get_all_species(frames).tolist()
    calculator.update_hyperparameters(**hypers)
    return np.concatenate(Parallel(n_jobs=2)(delayed(retrieve_features)(calculator, chunk)\
                                              for chunk in grouper(blocksize,frames)))


In [42]:
paramdict = {}
for param in zip(space,res_gp.x):
    paramdict[param[0].name] = param[1]

In [43]:
paramdict

{'alpha': 0.0033970924303804028,
 'gaussian_sigma_constant': 0.05557628804633241,
 'interaction_cutoff': 3.6601119046746073}

In [28]:
import cProfile

In [30]:
cProfile.run("gp_minimize(soap_objective, space, n_calls=10, random_state=0)")

         551440 function calls (545703 primitive calls) in 168.840 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2935    0.003    0.000    0.030    0.000 <__array_function__ internals>:2(all)
      117    0.000    0.000    0.009    0.000 <__array_function__ internals>:2(allclose)
      141    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(alltrue)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(amax)
       19    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(amin)
      422    0.000    0.000    0.005    0.000 <__array_function__ internals>:2(any)
      145    0.000    0.000    0.002    0.000 <__array_function__ internals>:2(append)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(argmax)
     2014    0.001    0.000    0.004    0.000 <__array_function__ internals>:2(argmin)
       13    0.000    0.000    0.002    0.000 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from skopt.plots import plot_convergence


plot_convergence(res_gp)
plt.xlim(3,50)
plt.ylim(0.8,1.)

In [ ]:
reg.

In [41]:
res_gp.x

[0.0033970924303804028, 0.05557628804633241, 3.6601119046746073]

In [26]:
np.sqrt(0.8)

0.8944271909999159

In [12]:

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))


In [54]:
list(split(range(10),3))

[range(0, 4), range(4, 7), range(7, 10)]

In [48]:
help(divmod)

Help on built-in function divmod in module builtins:

divmod(x, y, /)
    Return the tuple (x//y, x%y).  Invariant: div*y + mod == x.



In [43]:
from joblib import Parallel, delayed

In [44]:

from joblib import Parallel, delayed

def split(a, n):
    #splits a list into n chunks, fails if len(a) is 0
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

def get_features(frames,calculator,hypers):
    calculatorinstance = calculator(**hypers)
    print("worker spawned")
    return calculatorinstance.transform(frames).get_features(calculatorinstance)

def get_features_in_parallel(frames,calculator,hypers,blocks=4):
    """helper function that returns the features of a calculator (from calculator.transform())
       in parallel
    """
    
    #block is necessary to ensure that shape of the chunks is equal
    hypers["expansion_by_species_method"] = "user defined"
    hypers["global_species"] = [1,6,7,8,16] #replace by get_atomic_species functions
    
    features = Parallel(n_jobs=blocks)(delayed(get_features)(frame_block, calculator,  hypers) for frame_block in split(frames,blocks))
    return np.concatenate(features)

In [24]:
Parallel(n_jobs=2)(delayed(np.sqrt)(i ** 2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [77]:
train_structures

[Atoms(symbols='C28H56N24O8', pbc=True, cell=[[8.609441092, 0.0, 0.0], [0.0, 12.5135557345, 0.0], [-3.0746724783, 0.0, 10.1224900978]], center_atoms_mask=..., cs_iso=..., cs_tensor=...),
 Atoms(symbols='C32H40N16S8', pbc=True, cell=[[8.26976045, 0.0, 0.0], [0.0, 9.2331710029, 0.0], [-0.511906441615, 0.0, 13.386310859]], center_atoms_mask=..., cs_iso=..., cs_tensor=...),
 Atoms(symbols='C56H72N24', pbc=True, cell=[[6.569558028, 0.0, 0.0], [0.0, 13.5715177958, 0.0], [-0.652429377319, 0.0, 14.9911401597]], center_atoms_mask=..., cs_iso=..., cs_tensor=...),
 Atoms(symbols='C72H64N16O24', pbc=True, cell=[[24.006144864, 0.0, 0.0], [0.0, 3.67582090158, 0.0], [-3.31289600352, 0.0, 18.8654449967]], center_atoms_mask=..., cs_iso=..., cs_tensor=...),
 Atoms(symbols='C36H40N8O16', pbc=True, cell=[[11.966453176, 0.0, 0.0], [0.0, 6.81872434875, 0.0], [-5.69740785389, 0.0, 11.9480846704]], center_atoms_mask=..., cs_iso=..., cs_tensor=...),
 Atoms(symbols='C40H36N4O16', pbc=True, cell=[8.029036468, 8.

In [28]:
import time

In [45]:
start_time = time.time()
_ = get_features_in_parallel(train_structures,calculator=SOAP,hypers=hypers,blocks=2)
print("--- %s seconds ---" % (time.time() - start_time))

--- 11.242979288101196 seconds ---
